# 🚀 EnfoadsIA - Backend Production (T4 GPU) PRO v3.0

Este notebook ejecuta el ecosistema completo de **EnfoadsIA** con **LivePortrait** y **Media Enhancement**.

## 📋 Instrucciones
1. Asegúrate de tener GPU habilitada: `Runtime > Change runtime type > T4 GPU`
2. Ejecuta las celdas en orden
3. **Inicia sesión** en HuggingFace para descargar los modelos premium

---

In [ ]:
# @title 🛠️ 1. Configuración del Entorno
import os
import sys
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
REPO_DIR = Path("/content/FoadsIA")
BACKEND_DIR = REPO_DIR / "backend"

def run_command(cmd, description):
    print(f"⚙️  {description}...")
    subprocess.run(cmd, shell=True, check=True)

os.chdir("/content")
if not REPO_DIR.exists():
    run_command(f"git clone {REPO_URL}", "Clonando Repositorio")
else:
    os.chdir(REPO_DIR)
    run_command("git pull", "Actualizando Repositorio")

os.chdir(BACKEND_DIR)
run_command("apt-get update -qq && apt-get install -y -qq ffmpeg libsm6 libxext6 libgl1", "Dependencias del Sistema")
run_command("pip install -q -r requirements.txt", "Dependencias de Python")
run_command("pip install -q huggingface_hub gradio pyyaml omegaconf coloredlogs edge-tts faster-whisper", "Dependencias adicionales")

print("✅ Entorno base configurado.")

In [ ]:
# @title 🔐 1.1. Autenticación (HuggingFace & CivitAI)
from huggingface_hub import login
import os

HF_TOKEN = "" # @param {type:"string"}
CIVITAI_API_KEY = "" # @param {type:"string"}

if HF_TOKEN:
    login(token=HF_TOKEN, add_to_git_credential=True)
else:
    print("⚠️  HF_TOKEN requerido para descargar pesos de LivePortrait automáticamente.")

if CIVITAI_API_KEY:
    os.environ['CIVITAI_API_KEY'] = CIVITAI_API_KEY
    print("✅ CivitAI Key guardada.")

In [ ]:
# @title 🎬 1.2. Configuración de LivePortrait
print("🚀 Instalando LivePortrait y descargando pesos (~700MB)...")
!python setup_liveportrait.py

In [ ]:
# @title 📥 1.3. Pre-descarga de Modelos base (SDXL, Whisper)
!python preload_models.py

In [ ]:
# @title 🚀 2. Iniciar Servidor Público
from pyngrok import ngrok
import time

AUTH_TOKEN = "" # @param {type:"string"}
PORT = 5000

if AUTH_TOKEN:
    ngrok.set_auth_token(AUTH_TOKEN)

ngrok.kill()
tunnel = ngrok.connect(PORT, bind_tls=True)
print(f"\n📡 URL DEL BACKEND: {tunnel.public_url}\n")

print("🚀 Iniciando servidor...")
!python app.py